### Задание 

Обучите нейронную сеть решать шифр цезаря.   
Алгоритм следующий :   
1.Написать алгоритм шифра цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп).  
2.Сделать нейронную сеть.  
3.Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза).  
4.Проверить качество.  

In [1]:
import pandas as pd
import time
import torch
import re
from google.colab import files

In [2]:
file = files.upload()

Saving data.csv to data.csv


In [3]:
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [4]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [5]:
# очистим текст от лишних знаков
text = [[c for c in re.sub('[^a-z ]', ' ', ph) ] for ph in phrases if type(ph) is str]

In [6]:
text[1]

['l', 'e', 'e', ' ', 'm', 'u', 'r', ' ', 'l', 'e', 'e', ' ', 'm', 'u', 'r']

In [7]:
CHARS = list('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
INDEX_TO_CHAR_CESAR = [w for w in CHARS]
CHAR_TO_INDEX = {w:i for i, w in enumerate(INDEX_TO_CHAR)}
CHAR_TO_INDEX_CESAR = {w:i for i, w in enumerate(INDEX_TO_CHAR_CESAR)}

In [8]:
def generate_cesar(text, shift=3):
    ''' Cesar algorythm '''
    assert text is not None
    new_text = []
    for char in text:
        if type(text) == str:
            idx = CHAR_TO_INDEX[char]
            if idx + shift < len(CHAR_TO_INDEX):
                new_idx = idx + shift
            else:
                new_idx = idx + shift - len(CHAR_TO_INDEX)
            new_char = INDEX_TO_CHAR[new_idx]
            new_text.append(new_char)
        elif type(text) == list:
            for j in char:
                idx = CHAR_TO_INDEX[j]
                if idx + shift < len(CHAR_TO_INDEX):
                    new_idx = idx + shift
                else:
                    new_idx = idx + shift - len(CHAR_TO_INDEX)
                new_char = INDEX_TO_CHAR[new_idx]
                new_text.append(new_char)
    #return ''.join(new_text)
    return new_text

In [9]:
generated_cesar_text = []
for i in range(len(text)):
    generated_cesar_text.append( generate_cesar(text[i], shift=3) )

In [10]:
print(text[1])
print(generated_cesar_text[1])

['l', 'e', 'e', ' ', 'm', 'u', 'r', ' ', 'l', 'e', 'e', ' ', 'm', 'u', 'r']
['o', 'h', 'h', 'b', 'p', 'x', 'u', 'b', 'o', 'h', 'h', 'b', 'p', 'x', 'u']


# Сеть

In [12]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
        self.rnn = torch.nn.RNN(28, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, 28)
        
    def forward(self, sentences, state=None):
        embds = self.embeddings(sentences)
        out, new_state = self.rnn(embds, state)
        result = self.linear(out)
        return result, new_state

In [13]:
model = Network()

In [14]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [15]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
Y = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(zip(text[i], generated_cesar_text[i])):
        if j >= MAX_LEN:
            break
        # на вход сети - зашифрованный поток
        # на выход - дешифрованны поток 
        X[i, j] = CHAR_TO_INDEX.get(w[1], CHAR_TO_INDEX['none'])
        Y[i, j] = CHAR_TO_INDEX.get(w[0], CHAR_TO_INDEX['none'])

In [16]:
X[:1]

tensor([[16,  4, 10, 10, 12,  8,  2, 15, 18, 18, 14,  2, 26, 11,  4, 23, 22,  2,
         23, 11,  4, 23,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [17]:
Y[:1]

tensor([[13,  1,  7,  7,  9,  5, 27, 12, 15, 15, 11, 27, 23,  8,  1, 20, 19, 27,
         20,  8,  1, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [18]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    # вход - защифрованная фраза
    # выход - дешифрованная фраза
    for i in range(int(len(X) / 100)):
        X_batch = X[i * 100:(i + 1) * 100]
        Y_batch = Y[i * 100:(i + 1) * 100].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 3.521, Train loss: 1.257
Epoch 1. Time: 3.339, Train loss: 0.412
Epoch 2. Time: 3.429, Train loss: 0.224
Epoch 3. Time: 3.609, Train loss: 0.147
Epoch 4. Time: 3.447, Train loss: 0.107
Epoch 5. Time: 3.429, Train loss: 0.084
Epoch 6. Time: 3.388, Train loss: 0.069
Epoch 7. Time: 3.413, Train loss: 0.059
Epoch 8. Time: 3.518, Train loss: 0.052
Epoch 9. Time: 3.436, Train loss: 0.047


In [19]:
def generate_sentence(sentence):
    state = None
    for i in range( len(sentence) ):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        result, state = model.forward( X, state )
        index = result.argmax()
        sentence.append(INDEX_TO_CHAR[index])
    print(''.join(sentence))

In [20]:
test_sentence = 'hello black'
encrypted_test_sentence = generate_cesar(test_sentence)
print(f'test_sentence : {test_sentence}')
print(f'encrypted_test_sentence : {encrypted_test_sentence}')

test_sentence : hello black
encrypted_test_sentence : ['k', 'h', 'o', 'o', 'r', 'b', 'e', 'o', 'd', 'f', 'n']


In [21]:
generate_sentence(sentence=encrypted_test_sentence)

khoorbeodfnhello black
